In [363]:
import pandas as pd
import numpy
from pulp import *

# Linear Programming Model

In [364]:
prob=LpProblem("Minimize",LpMinimize)

In [365]:
#time taken to finish each work
w1=LpVariable("w1",lowBound=1)
w2=LpVariable("w2",lowBound=1)
w3=LpVariable("w3",lowBound=1)
w4=LpVariable("w4",lowBound=1)
w5=LpVariable("w5",lowBound=1)
w6=LpVariable("w6",lowBound=1)
w7=LpVariable("w7",lowBound=1)
w8=LpVariable("w8",lowBound=1)
w9=LpVariable("w9",lowBound=1)
w10=LpVariable("w10",lowBound=1)
w11=LpVariable("w11",lowBound=1)
w12=LpVariable("w12",lowBound=1)
w13=LpVariable("w13",lowBound=1)
w14=LpVariable("w14",lowBound=1)
w15=LpVariable("w15",lowBound=1)

In [366]:
#objective function is find minimum total time required to finish each work
prob+=(w1+w2+w3+w4+w5+w6+w7+w8+w9+w10+w11+w12+w13+w14+w15)

In [367]:
#prob.solve()

# Solution

## Data CleanUp:

In [368]:
def data(filename):
    df=pd.read_csv(filename)
    df=df.drop(columns=['Unnamed: 0'])
    df=df.rename(columns={"breaks between works": "breaks"}, errors="raise")
    breaks = df["breaks"].to_numpy()
    df=df.drop(columns=['breaks'])
    return df, breaks


# Heuristic Approach to Solve Linear Programming:
 This solution is built using few heurisitcs, the search space is explored to obtain the best possible answer.

 Each column is converted into an array, each column represents timings for each work to a specific machine.
   The algorithm starts by assigning work in an ascending sequence, and at each step it is ensured that the timing selected is minimal from the list obtained in step given above and we try to minimize the break times by ensuring that the consecutive works are not allocated to the same machine.
   The result is stored in a dictionary, keys being the work id and the values are the machines that a work is assigned to.
   Once all the works in the problem statement are allocated to some machine, we have to consider the break times.
   Loop through the allocations dictionary to check whether two consecutive works are allocated to same machines.
   

In [369]:
#function to find allocation, the parameter passed is the pandas dataframe.
def find_allocation(filename):
    #dictionary to store the final allocation.
    allocations={}
    #runtime obtained from the allocations.
    total_time=0
    #list for search space.
    l=[]
    #a counter for operations.
    c=0
    df,breaks=data(filename)
    
    #loop through all the works to be allocated to machines.
    for i in df.columns:
        c=c+1
        #convert the timings to list
        values = df[i].to_numpy()
        #find the minimum timing
        result = numpy.where(values == numpy.amin(values))
        #add the minimum time to the result
        total_time += numpy.amin(values)
        #if the first or the last works from  the list
        if(c==1 or c==len(df.columns)):
            allocations[i] = ((result[0][0]+1))
        #if the work is b/w last and first
        if(l):
            #below is the heuristic to perform allocation
            find=l[len(l)-1]
            work_assigned=0
            for k in find:
                for item in result[0]:
                    if(k==item):
                        continue
                    else:
                        work_assigned=1
                        #work is allocated to the machine which is (item+1).
                        allocations[i] = ((item+1))
                        #append the result in l for further processing.
                        l.append(result[0])
        
        l.append(result[0])   
    
    #after the allocation is found, aim is to find the break times required and add to total time.

    counter=0
    previous_machine=None
    #loop throught the result.
    for i in allocations:
        #print(i)
        #check if allocation is first allocation.
        if(counter==0):
            counter+=1
            previous_machine=allocations[i]
            allocations[i]="Machine: "+str(allocations[i])
            continue
        #check if the allocation is similar to previous allocation, if yes, add the break time to total time.
        if(allocations[i]==previous_machine):
            break_time=breaks[allocations[i]]
            total_time+=break_time
        previous_machine=allocations[i]
        #edit allocation to make it more readable.
        allocations[i]="Machine: "+str(allocations[i])
        
    return allocations,total_time



In [370]:
#call the method to obtain result, just pass the data through.
filename="WorkDistribution.csv"
allocations,total_time=find_allocation(filename)

In [371]:
total_time

246

In [372]:
allocations

{'Work 1': 'Machine: 3',
 'Work 2': 'Machine: 5',
 'Work 3': 'Machine: 3',
 'Work 4': 'Machine: 1',
 'Work 5': 'Machine: 5',
 'Work 6': 'Machine: 1',
 'Work 7': 'Machine: 5',
 'Work 8': 'Machine: 8',
 'Work 9': 'Machine: 7',
 'Work 10': 'Machine: 5',
 'Work 11': 'Machine: 1',
 'Work 12': 'Machine: 6',
 'Work 13': 'Machine: 8',
 'Work 14': 'Machine: 7',
 'Work 15': 'Machine: 9'}